In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
path = '../input/final-data/tra_data.npy'
path

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
train.head()

In [ ]:
path='/kaggle/input/x-data/tra_data.npy'
path

In [ ]:
p = os.path.join('../kaggle/input/final-data', 'tra_data.npy')
p

In [ ]:
x=np.load(path)
x.shape

In [ ]:
x.shape

In [ ]:
train.head()
y = train.iloc[:, 1:]
y = y.to_numpy()

y.shape

In [ ]:
y[:5]

In [ ]:
y.sum(axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
x_train ,x_test, y_train, y_test = train_test_split(x, y , test_size=0.20, random_state=42)

In [ ]:
del x

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# x_train_final = x_train.reshape(x_train.shape[0], -1)
x_train_final = x_train /255.0
x_train_final.shape

In [ ]:
# x_test_final = x_test.reshape(x_test.shape[0], -1)
x_test_final = x_test /255.0
x_test_final.shape

In [ ]:
y_train_final = y_train
y_train_final.shape

In [ ]:
y_test_final = y_test
y_test_final.shape

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
img_size=224

In [ ]:
vgg = VGG16(input_shape=(img_size, img_size, 3), weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# our layers - you can add more if you want
X = Flatten()(vgg.output)
X = Dense(4096, activation='relu')(X)
X = Dropout(0.2)(X)
X = Dense(4096, activation='relu')(X)
X = Dropout(0.2)(X)
X = Dense(4096, activation='relu')(X)

prediction = Dense(4, activation='softmax')(X)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=.1,
                             fill_mode='nearest',
                             shear_range=0.1,
                             rescale=1/255,
                             brightness_range=[0.5, 1.5])


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',cooldown=1, verbose=1,min_lr=0.000001,patience=6)

In [ ]:
best_model = ModelCheckpoint('VGG16.h5', monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True,mode='min')

In [ ]:
x_train_final.shape

In [ ]:
history = model.fit_generator(datagen.flow(x_train_final, y_train_final),
                              epochs=300,
#                               steps_per_epoch=x_train_final.shape[0] // 24,
                              verbose=1,
                              callbacks=[reduce_lr,best_model],
                              validation_data=datagen.flow(x_test_final, y_test_final),
#                               validation_steps=x_test_final.shape[0]//24
                              )

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
x_test_final.shape

In [ ]:
y_test_final.shape

In [ ]:
history.history.keys()

In [ ]:
from keras.app